In [38]:
from pymol import cmd
import py3Dmol

from vina import Vina

from openbabel import pybel

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

from meeko import MoleculePreparation
from meeko import obutils

import MDAnalysis as mda
from MDAnalysis.coordinates import PDB

import prolif as plf
from prolif.plotting.network import LigNetwork


import sys, os
sys.path.insert(1, 'utilities/')
from utils import fix_protein, getbox, generate_ledock_file, pdbqt_to_sdf, dok_to_sdf


os.system("pythonsh")
import prepare_receptor
import warnings
warnings.filterwarnings("ignore")
%config Completer.use_jedi = False


dok_to_sdf(dok_file='lx_0.dok',output='lx_0.sdf')
dok_to_sdf(dok_file='lx_1.dok',output='lx_1.sdf')
dok_to_sdf(dok_file='lx_2.dok',output='lx_2.sdf')

ModuleNotFoundError: No module named 'MolKit'

In [31]:


view = py3Dmol.view()
view.removeAllModels()
view.setViewStyle({'style':'outline','color':'black','width':0.1})

view.addModel(open('CHAIN_A.pdb','r').read(),format='pdb')
Prot=view.getModel()
Prot.setStyle({'cartoon':{'arrows':True, 'tubes':True, 'style':'oval', 'color':'white'}})
view.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


view.addModel(open('lx_0.mol2','r').read(),format='mol2')
ref_m = view.getModel()
ref_m.setStyle({},{'stick':{'colorscheme':'magentaCarbon','radius':0.2}})

results=Chem.SDMolSupplier('lx_0.sdf')


p=Chem.MolToMolBlock(results[0])

print('Reference: Magenta | LeDock Pose: Cyan')
print ('Pose: {} | Score: {}'.format(results[0].GetProp('Pose'),results[0].GetProp('Score')))

view.addModel(p,'mol')
x = view.getModel()
x.setStyle({},{'stick':{'colorscheme':'cyanCarbon','radius':0.2}})

view.zoomTo()
view.show()



Reference: Magenta | LeDock Pose: Cyan
Pose: 8 | Score: -6.17


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [20]:
fix_protein(filename='CHAIN_A.pdb',addHs_pH=7.4,output='CHAIN_A_fixed.pdb',try_renumberResidues=False)

prot = mda.Universe("CHAIN_A_fixed.pdb")
print(prot.atoms[426])

<Atom 427: HD13 of type H of resname ILE, resid 26 and segid A and altLoc >


In [17]:
# load protein




#prot = mda.Universe("CHAIN_A.pdb",guess_bonds=True)
prot = mda.Universe("CHAIN_A_fixed.pdb")
prot = plf.Molecule.from_mda(prot)
prot.n_residues

# load ligands
path = str('lx_0.sdf')
lig_suppl = list(plf.sdf_supplier(path))
# generate fingerprint
fp = plf.Fingerprint()
fp.run_from_iterable(lig_suppl, prot)
results_df = fp.to_dataframe(return_atoms=True)
results_df

[23:24:26] Explicit valence for atom # 428 H, 2, is greater than permitted


AtomValenceException: Explicit valence for atom # 428 H, 2, is greater than permitted